<a href="https://colab.research.google.com/github/SeojinSeojin/deep-learning_challenge/blob/main/TermProject/Pokemon_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

**Train Dataset 추가**

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d thedagger/pokemon-generation-one

In [ ]:
!unzip pokemon-generation-one.zip

In [ ]:
import os

directory = "./dataset"
pokemons = []
for file in os.listdir(directory):
  pokemons.append(file)
# print(pokemons[:10], end=", ")
indexes = [(x) for x in range(len(pokemons))]

pokemons_with_indexes = dict(zip(pokemons, indexes))
print(pokemons_with_indexes)
print(len(pokemons))

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

dataset = []
count = 0
for file in os.listdir(directory):
  path = os.path.join(directory, file)
  for image in os.listdir(path):
    if image[-3:] == 'jpg':
      image = load_img(os.path.join(path, image), target_size=(128,128))
      image = img_to_array(image)
      image = image/255.0
      dataset.append([image, pokemons_with_indexes[file]])

**Test Dataset 추가**

In [ ]:
!kaggle datasets download -d hlrhegemony/pokemon-image-dataset

In [ ]:
!unzip pokemon-image-dataset.zip

In [ ]:
reverse_mapping=dict(zip(indexes,pokemons)) 

def mapper(value):
    return reverse_mapping[value]

In [ ]:
directory2 = "./images"

testset = []
for file in os.listdir(directory2):
  if file in pokemons:
    path = os.path.join(directory2, file)
    for image in os.listdir(path):
      image = load_img(os.path.join(path, image), target_size=(128,128))
      image = img_to_array(image)
      image = image/255.0
      testset.append([image, pokemons_with_indexes[file]])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, array_to_img

array_to_img(testset[2][0])

In [ ]:
array_to_img(dataset[100][0])

In [ ]:
data,dlabels=zip(*dataset)
test,tlabels=zip(*testset)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

data = np.array(data)
test = np.array(test)
dlabels = np.array(list(dlabels))
tlabels = np.array(list(tlabels))
print(data.shape, len(dlabels), test.shape, len(tlabels))
print(len(set(dlabels)))

**학습**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (Dense, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout)

model = keras.models.Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation="relu"),
    Flatten(),
    Dropout(rate=0.2),
    Dense(256, activation="relu"),
    Dense(516, activation="relu"),
    BatchNormalization(),
    Dense(516, activation="relu"),
    Dense(150, activation="sigmoid")
])

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(data, dlabels, epochs=200, batch_size=200)

**테스트**

In [ ]:
loss_and_metrics = model.evaluate(test, tlabels, batch_size=100)
print('')
print('loss_and_metrics : ' + str(loss_and_metrics))

model.save("pokemon_model.h5")

In [ ]:
from PIL import Image
import requests


url = "https://pbs.twimg.com/profile_images/1116329667147919360/4-lamrzL_400x400.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image = image.resize((128, 128))

image=img_to_array(image)
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)

value2 = np.argsort(np.max(prediction, axis=0))[-2]
value3 = np.argsort(np.max(prediction, axis=0))[-3]
value4 = np.argsort(np.max(prediction, axis=0))[0]

print(f"Prediction is {mapper(value)}.")
print(f"Second Prediction is {mapper(value2)}.")
print(f"Third Prediction is {mapper(value3)}")
print(f"Least Prediction is {mapper(value4)}")

In [ ]:
import json

with open("pokemons.json", "w", encoding="utf-8") as make_file:
  json.dump(reverse_mapping, make_file, indent="\t")